In [1]:
import numpy as np
from numpy.linalg import solve, norm



# Base working

In [6]:
def primal_dual_ipm(c, A, b, max_iter=10000, tol=1e-10, mu=0.1):
    """
    Solve Linear Programming problem using Primal-Dual Path-Following Interior Point Method
    
    minimize    c^T x
    subject to  Ax = b
                x >= 0
    
    Parameters:
    -----------
    c : array_like
        Objective function coefficients
    A : array_like
        Constraint matrix
    b : array_like
        Right-hand side of constraints
    max_iter : int, optional
        Maximum number of iterations
    tol : float, optional
        Tolerance for convergence
    mu : float, optional
        Path parameter (between 0 and 1)
    
    Returns:
    --------
    x : ndarray
        Optimal primal solution
    y : ndarray
        Optimal dual solution
    s : ndarray
        Optimal slack variables
    """
    
    # Convert inputs to numpy arrays
    c = np.array(c, dtype=float)
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    
    m, n = A.shape
    
    # Initialize variables
    x = np.ones(n)  # Primal variables
    y = np.zeros(m)  # Dual variables
    s = np.ones(n)  # Slack variables
    
    # Main loop
    for iteration in range(max_iter):
        # Current duality measure
        mu_current = np.dot(x, s) / n
        
        # Check convergence
        primal_residual = np.dot(A, x) - b
        dual_residual = np.dot(A.T, y) + s - c
        complementarity = x * s
        
        if (norm(primal_residual) < tol and 
            norm(dual_residual) < tol and 
            norm(complementarity) < tol):
            break
            
        # Compute search direction
        X = np.diag(x)
        S = np.diag(s)
        
        # Form the KKT matrix
        zero_m = np.zeros((m, m))
        row1 = np.hstack([S, np.zeros((n, m)), X])
        row2 = np.hstack([A, np.eye(m), np.zeros((m, n))])
        row3 = np.hstack([np.zeros((n, n)), A.T, np.eye(n)])
        KKT = np.vstack([row1, row2, row3])
        
        # Compute right-hand side
        sigma = mu  # Centering parameter
        rc = -complementarity + sigma * mu_current
        rp = -primal_residual
        rd = -dual_residual
        rhs = np.concatenate([rc, rp, rd])
        
        # Solve KKT system
        solution = solve(KKT, rhs)
        dx = solution[:n]
        dy = solution[n:n+m]
        ds = solution[n+m:]
        
        # Line search
        alpha_primal = 0.99995 * min(1, min(-x[dx < 0] / dx[dx < 0]) if any(dx < 0) else 1)
        alpha_dual = 0.99995 * min(1, min(-s[ds < 0] / ds[ds < 0]) if any(ds < 0) else 1)
        
        # Update variables
        x += alpha_primal * dx
        y += alpha_dual * dy
        s += alpha_dual * ds
    
    return x, y, s

# Example usage
if __name__ == "__main__":
    # Example problem:
    # minimize    -x1 - x2
    # subject to   x1 + x2 = 1
    #              x1, x2 >= 0
    
    # c = np.array([-1, -1])
    # A = np.array([[1, 1]])
    # b = np.array([1])
   
    # Alternate problem
    A = np.array([[1, 2, 3], [4, 5, 6]])
    b = np.array([7, 8])
    c = np.array([1, 2, 3])
    
    x, y, s = primal_dual_ipm(c, A, b)
    
    print("Optimal solution:")
    print(f"x = {x}")
    print(f"y = {y}")
    print(f"s = {s}")
    print(f"Objective value = {np.dot(c, x)}")

Optimal solution:
x = [0.         0.         1.53333333]
y = [ 23998.25475387 -11998.62737693]
s = [23997.25475387 11998.62737693     0.        ]
Objective value = 4.600000000000727


# Experimantal

In [17]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg
from scipy.sparse.linalg import lsqr
import warnings

class SparseCholesky:
    """Specialized Cholesky factorization for sparse matrices"""
    def __init__(self, A):
        self.n = A.shape[0]
        # Convert to CSC format for efficient column operations
        if not sparse.isspmatrix_csc(A):
            A = sparse.csc_matrix(A)
            
        # Add small diagonal perturbation for numerical stability
        diagonal_perturbation = 1e-8
        A = A + sparse.diags([diagonal_perturbation] * A.shape[0])
        
        # Store matrix for later use
        self.A = A
        
        # Create LU factorization using SuperLU
        self.lu = sparse.linalg.splu(A)
            
    def solve(self, b):
        """Solve system Ax = b using the LU factorization"""
        return self.lu.solve(b)

def adaptive_step_length(x, s, dx, ds, eta=0.95):
    """Compute adaptive step length with Mehrotra predictor-corrector"""
    # Compute maximum allowable step lengths
    alpha_primal = -1/min(dx[dx < 0] / x[dx < 0]) if any(dx < 0) else 1
    alpha_dual = -1/min(ds[ds < 0] / s[ds < 0]) if any(ds < 0) else 1
    
    # Initial step lengths
    alpha_p = min(1, eta * alpha_primal)
    alpha_d = min(1, eta * alpha_dual)
    
    # Compute predicted reduction in complementarity
    old_comp = np.dot(x, s)
    new_x = x + alpha_p * dx
    new_s = s + alpha_d * ds
    new_comp = np.dot(new_x, new_s)
    
    # Adjust step lengths based on complementarity reduction
    if new_comp > (1 - 0.01 * min(alpha_p, alpha_d)) * old_comp:
        gamma = 0.9  # More conservative
    else:
        gamma = 0.95  # More aggressive
        
    return gamma * alpha_p, gamma * alpha_d

def solve_iterative_refinement(A, b, x0, max_refinements=3, tol=1e-8):
    """Solve linear system with iterative refinement using LSQR"""
    x = x0.copy()
    
    for _ in range(max_refinements):
        # Compute residual
        r = b - A @ x
        
        # Check if residual is small enough
        if np.linalg.norm(r) < tol:
            break
            
        # Solve correction equation using LSQR
        dx, *_ = lsqr(A, r, atol=tol, btol=tol)
        
        # Update solution
        x = x + dx
        
    return x

def advanced_primal_dual_ipm(c, A, b, max_iter=100, tol=1e-8, mu=0.1):
    """
    Enhanced Primal-Dual Interior Point Method with advanced features
    """
    # Convert inputs to sparse format if not already
    if not sparse.issparse(A):
        A = sparse.csc_matrix(A)
    
    m, n = A.shape
    
    # Initialize variables
    x = np.ones(n)
    y = np.zeros(m)
    s = np.ones(n)
    
    # Initial mu value
    mu_current = np.dot(x, s) / n
    
    for iteration in range(max_iter):
        # Compute residuals
        primal_residual = A @ x - b
        dual_residual = A.T @ y + s - c
        complementarity = x * s
        
        # Check convergence
        if (np.linalg.norm(primal_residual) < tol and 
            np.linalg.norm(dual_residual) < tol and 
            np.linalg.norm(complementarity) < tol):
            break
            
        # Form the normal equations matrix
        D = sparse.diags(x / s)
        M = A @ D @ A.T
        
        # Initialize solver
        chol = SparseCholesky(M)
        
        # Compute affine scaling direction
        rc = -complementarity
        rx = -dual_residual
        ry = -primal_residual
        
        # Solve normal equations
        dy = chol.solve(ry - A @ D @ rx)
        dx = D @ (A.T @ dy - rx)
        ds = -(dual_residual + A.T @ dy)
        
        # Compute adaptive step lengths
        alpha_p, alpha_d = adaptive_step_length(x, s, dx, ds)
        
        # Update variables
        x += alpha_p * dx
        y += alpha_d * dy
        s += alpha_d * ds
        
        # Update mu
        mu_current = np.dot(x, s) / n
        
    return x, y, s

# Example usage
if __name__ == "__main__":
    # Test problem
    c = np.array([-1, -1])
    A = sparse.csc_matrix([[1, 1]])
    b = np.array([1])

    # Alternate problem
    A = np.array([[1, 2, 3], [4, 5, 6]])
    b = np.array([7, 8])
    c = np.array([1, 2, 3])
    
    x, y, s = advanced_primal_dual_ipm(c, A, b)
    
    print("Optimal solution:")
    print(f"x = {x}")
    print(f"y = {y}")
    print(f"s = {s}")
    print(f"Objective value = {np.dot(c, x)}")

Optimal solution:
x = [4.02481087e-09 7.25628689e-08 1.53429228e+00]
y = [ 2.02010093e+10 -1.01005047e+10]
s = [2.02010093e+10 1.01005047e+10 2.33434679e-02]
Objective value = 4.6028769747561915


# Pulp Working

In [4]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Create the problem
problem = LpProblem("Simple_MIP_Problem", LpMaximize)

# Define decision variables
x = LpVariable("x", lowBound=0, cat="Continuous")  # x is an integer variable
y = LpVariable("y", lowBound=0, cat="Continuous")  # y is a continuous variable
z = LpVariable("z", lowBound=0, cat="Continuous")  # z is a continuous variable


# Define the objective function
problem += 1 * x + 2 * y + 3 * z, "Objective"

# Define the constraints
problem +=  1 * x + 2 * y + 3 * z <= 7, "Constraint_1"
problem +=  4 * x + 5 * y + 6 * z <= 8, "Constraint_2"

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)
print("Optimal Solution:")
print(f"x = {x.varValue}")
print(f"y = {y.varValue}")
print(f"z = {z.varValue}")

print("Objective value =", problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/vivekchaudhary/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/17a0da27f4ea4ef3bafd9226bd28aacb-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/17a0da27f4ea4ef3bafd9226bd28aacb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 17 RHS
At line 20 BOUNDS
At line 21 ENDATA
Problem MODEL has 2 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
0  Obj -0 Dual inf 6.8999997 (3)
0  Obj -0 Dual inf 6.8999997 (3)
1  Obj 4
Optimal - objective value 4
Optimal objective 4 - 1 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock